In [ ]:
from main import main

import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
from matplotlib import rcParams
import time

size = main()

rcParams['animation.embed_limit'] = 1000  # Größere Animationen einbetten

# Lade die Simulationsdaten
with open("simulation_data.json", "r") as file:
    data = json.load(file)

# Extrahiere die Zeitschritte und Anzahl der Körper
timesteps = sorted(map(int, data.keys()))
num_bodies = len(data[str(timesteps[0])]["positions"][0])  # Anzahl der Körper

# Konvertiere JSON-Daten in NumPy-Arrays
coordinates = [
    np.array(data[str(t)]["positions"]).T for t in timesteps
]

# Berechne die Weltgröße für die Achsen-Skalierung
all_x = [x for t in timesteps for x in data[str(t)]["positions"][0]]
all_y = [y for t in timesteps for y in data[str(t)]["positions"][1]]
all_z = [z for t in timesteps for z in data[str(t)]["positions"][2]]

x_min, x_max = min(all_x), max(all_x)
y_min, y_max = min(all_y), max(all_y)
z_min, z_max = min(all_z), max(all_z)

# Initialisiere das 3D-Plot-Fenster
fig = plt.figure(figsize=(8, 8))
ax_scatter = fig.add_subplot(111, projection="3d")

scatter = ax_scatter.scatter([], [], [], s=50, c="blue", animated=True)

ax_scatter.set_xlim(size/2, size/2)
ax_scatter.set_ylim(size/2, size/2)
ax_scatter.set_zlim(size/2, size/2)

ax_scatter.set_xlabel("X-coordinate")
ax_scatter.set_ylabel("Y-coordinate")
ax_scatter.set_zlabel("Z-coordinate")
ax_scatter.set_title("3D-Simulation Animation")

# Update-Funktion für die Animation
def update(frame):
    """Aktualisiert die Positionen der Körper für jeden Zeitschritt."""
    points = coordinates[frame]
    scatter._offsets3d = (points[:, 0], points[:, 1], points[:, 2])  # X, Y, Z aktualisieren
    ax_scatter.set_title(f"Time Step {frame}")
    return scatter,

print("Creating animation...")
start_time_animation = time.time()

# Erstelle die Animation
ani = FuncAnimation(fig, update, frames=len(timesteps), blit=False, interval=100)

plt.close()  # Verhindert doppelte Anzeige

# Animation in Jupyter anzeigen
display(HTML(ani.to_jshtml()))

end_time_animation = time.time()
print(f"I took {round(end_time_animation - start_time_animation, 2)} seconds to create the animation.")


Time step 1:
Body 0 position: [  2.46858458 -20.96532166  25.44352642]
Body 1 position: [  2.46858458 -20.96532166  25.44352642]
Body 2 position: [-14.96968673   6.49482326 -22.52556938]
Body 3 position: [-14.96968673   6.49482326 -22.52556938]
Body 4 position: [ 28.81315591 -23.02831326  19.97659417]
Body 5 position: [ 28.81315591 -23.02831326  19.97659417]
Time step 2:
Body 0 position: [  2.66556606 -20.78234476  25.63511593]
Body 1 position: [  2.66556606 -20.78234476  25.63511593]
Body 2 position: [-14.85170498   6.6307324  -22.38171324]
Body 3 position: [-14.85170498   6.6307324  -22.38171324]
Body 4 position: [ 28.95374059 -22.83806363  20.13150582]
Body 5 position: [ 28.95374059 -22.83806363  20.13150582]
Time step 3:
Body 0 position: [  2.86254754 -20.59936785  25.82670544]
Body 1 position: [  2.86254754 -20.59936785  25.82670544]
Body 2 position: [-14.73372324   6.76664154 -22.23785709]
Body 3 position: [-14.73372324   6.76664154 -22.23785709]
Body 4 position: [ 29.09432528 -2

TypeError: float() argument must be a string or a real number, not 'NoneType'